# EnsLSTM
### Importing libraries

In [1]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split

from keras.optimizers import Adam
from keras.layers import Dropout
from keras.models import Sequential, Model
from keras.layers import LSTM, Dense, concatenate

### Loading the train dataset

In [2]:
water_consumption = pd.read_csv("train.csv")
water_consumption.head(10)

,Year,Month,Consumer_type,Consumption,Consumer_number,Installation_zone
0,2013,1,domestic,0,MOGV36480546611521,Installation_zone 1
1,2013,1,industrial,5,BECS02817768252637,Installation_zone 2
2,2013,1,domestic,6,VRFW65577141436242,Installation_zone 2
3,2013,1,domestic,1,QLLI18662653137621,Installation_zone 2
4,2013,1,domestic,13,HYUO61823402850645,Installation_zone 2
5,2013,1,industrial,27,FHMG62751338090488,Installation_zone 2
6,2013,1,industrial,5,APVF78863215212358,Installation_zone 2
7,2013,1,domestic,31,MXWL75757930683403,Installation_zone 2
8,2013,1,industrial,2,NVMY31359391120094,Installation_zone 2
9,2013,1,industrial,0,PZAN37359795617576,Installation_zone 2


In [3]:
train, test = train_test_split(water_consumption, test_size=0.2,
                               stratify=water_consumption['Consumer_type'], random_state=42)

## Model

In [4]:
class EnsLSTM():
  def __init__(self):
    self.networks = {}

  def fit(self, water_consumption):
    installation_groups = self.create_time_matrix_by_groups(water_consumption)

    print("Training ...")
    for group_name, group in installation_groups:
      print(', '.join(group_name))
      X_data, y_data = self.preprocess_data(group,
                                     target_column='Consumer_type',
                                     feature_column='Consumption')

      net = self.createIndLSTM(X_data.shape, len(group['Consumer_type'].unique()))
      net.fit(X_data, y_data, epochs=10, batch_size=32, validation_split=0.2)

      self.networks[', '.join(group_name)] = net

  def create_time_matrix_by_groups(self, water_consumption):
    print("Creating temporal matrix ...")

    water_consumption_months = self.agg_months(water_consumption,
     ['Consumer_type', 'Consumer_number', 'Installation_zone', 'Year'])

    water_consumption_years = self.agg_years(water_consumption_months,
     ['Consumer_type', 'Consumer_number', 'Installation_zone'])

    # installation_groups = water_consumption_years.groupby('Installation_zone')
    installation_groups = self.create_groups(water_consumption_years)

    return installation_groups

  def agg_months(self, water_consumption, group_features):
    all_months = list(range(1, 13))
    water_consumption_months = water_consumption.groupby(group_features).apply(
        lambda group: pd.Series({
            'Consumption': [group.loc[group['Month'] == month, 'Consumption'].values[0] if month in group['Month'].values else 0 for month in all_months]
        })
    ).reset_index()

    return water_consumption_months

  def agg_years(self, water_consumption_months, group_features):
    unique_years = water_consumption_months['Year'].unique()

    water_consumption_years = water_consumption_months.groupby(group_features).apply(
        lambda group: pd.Series({
            'Consumption': [group.loc[group['Year'] == year, 'Consumption'].values[0] if year in group['Year'].values else np.zeros(12) for year in unique_years]
        })
    ).reset_index()

    return water_consumption_years

  def create_groups(self, water_consumption_years):
    inst_consumer_types = water_consumption.groupby(['Consumer_type', 'Installation_zone']).size().reset_index(name='Count')

    inst_consumer_types = pd.get_dummies(inst_consumer_types, columns=['Consumer_type'], prefix='Consumer_type')
    inst_consumer_types = inst_consumer_types.groupby(['Installation_zone']).agg({
        'Count': 'sum',
        'Consumer_type_construction': 'sum',
        'Consumer_type_domestic': 'sum',
        'Consumer_type_industrial': 'sum',
        'Consumer_type_low income families': 'sum',
        'Consumer_type_rural commercial': 'sum',
        'Consumer_type_rural expansion': 'sum',
        'Consumer_type_rural domestic': 'sum'
      }).reset_index()

    inst_consumer_types['Consumer_types'] = inst_consumer_types.iloc[:, 2:].astype(str).apply(lambda row: ','.join(map(str, row)), axis=1)

    inst_consumer_types = inst_consumer_types.groupby(['Consumer_types']).agg({
        'Count': 'sum',
        'Installation_zone': lambda x: ', '.join(map(str, x))
      }).reset_index()

    inst_consumer_types['Installation_zone'] = inst_consumer_types['Installation_zone'].str.split(',')

    installation_groups = []

    for installations in inst_consumer_types['Installation_zone']:
      df_by_installations = water_consumption_years[water_consumption_years['Installation_zone'].isin(installations)]

      installation_groups.append((installations, df_by_installations))

    return installation_groups

  def preprocess_data(self, df, target_column=None, feature_column='Consumption', test_size=0.2, random_state=42):
    X = np.array(df[feature_column].tolist())
    # X = np.transpose(X, (0, 2, 1))

    if target_column == None:
      return X

    y = pd.get_dummies(df[target_column]).values

    return X, y

  def createIndLSTM(self, shape, num_labels):
    net = Sequential()
    net.add(LSTM(50, input_shape=(shape[1], shape[2])))
    net.add(Dense(num_labels, activation='softmax'))

    if num_labels > 1:
      net.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    else:
      net.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    return net

  def eval(self, water_consumption):
    installation_groups = self.create_time_matrix_by_groups(water_consumption)
    print("Test ...")

    right = 0
    total = 0
    for group_name, group in installation_groups:
      print(', '.join(group_name))
      X_data, y_data = self.preprocess_data(group,
                                     target_column='Consumer_type',
                                     feature_column='Consumption')


      y_pred_proba = self.networks[', '.join(group_name)].predict(X_data)

      y_data = np.argmax(y_data, axis=1)
      y_pred = np.argmax(y_pred_proba, axis=1)

      tmp_right = np.sum(y_data == y_pred)
      right += tmp_right
      total += len(y_data)

    return right / total

#### Train

In [5]:
network = EnsLSTM()
network.fit(train)

Creating temporal matrix ...
Training ...
Installation_zone 15,  Installation_zone 27,  Installation_zone 30
Epoch 1/10
1/1 [==============================] - 3s 3s/step - loss: 0.7805 - accuracy: 1.0000 - val_loss: 0.7644 - val_accuracy: 1.0000
Epoch 2/10
1/1 [==============================] - 0s 57ms/step - loss: 0.7494 - accuracy: 1.0000 - val_loss: 0.7468 - val_accuracy: 1.0000
Epoch 3/10
1/1 [==============================] - 0s 56ms/step - loss: 0.7195 - accuracy: 1.0000 - val_loss: 0.7297 - val_accuracy: 1.0000
Epoch 4/10
1/1 [==============================] - 0s 70ms/step - loss: 0.6909 - accuracy: 1.0000 - val_loss: 0.7131 - val_accuracy: 1.0000
Epoch 5/10
1/1 [==============================] - 0s 59ms/step - loss: 0.6633 - accuracy: 1.0000 - val_loss: 0.6968 - val_accuracy: 1.0000
Epoch 6/10
1/1 [==============================] - 0s 40ms/step - loss: 0.6367 - accuracy: 1.0000 - val_loss: 0.6808 - val_accuracy: 1.0000
Epoch 7/10
1/1 [==============================] - 0s 41ms/s

#### Test

In [6]:
acc = network.eval(test)

print("Acc: " + str(acc))

Creating temporal matrix ...
Test ...
Installation_zone 15,  Installation_zone 27,  Installation_zone 30
1/1 [==============================] - 1s 657ms/step
Installation_zone 17,  Installation_zone 21,  Installation_zone 32,  Installation_zone 44
6/6 [==============================] - 1s 4ms/step
Installation_zone 40,  Installation_zone 45
3/3 [==============================] - 1s 7ms/step
Installation_zone 20,  Installation_zone 37,  Installation_zone 43
2/2 [==============================] - 1s 10ms/step
Installation_zone 26,  Installation_zone 31
4/4 [==============================] - 1s 4ms/step
Installation_zone 13,  Installation_zone 18


5/5 [==============================] - 0s 4ms/step
Installation_zone 11,  Installation_zone 14,  Installation_zone 19,  Installation_zone 23,  Installation_zone 25,  Installation_zone 28,  Installation_zone 36,  Installation_zone 41,  Installation_zone 48,  Installation_zone 6,  Installation_zone 7
3/3 [==============================] - 0s 4ms/step
Installation_zone 42,  Installation_zone 8
4/4 [==============================] - 0s 4ms/step
Installation_zone 10
11/11 [==============================] - 0s 3ms/step
Installation_zone 33,  Installation_zone 47
3/3 [==============================] - 0s 8ms/step
Installation_zone 12,  Installation_zone 39,  Installation_zone 49,  Installation_zone 5,  Installation_zone 9
4/4 [==============================] - 0s 4ms/step
Installation_zone 16,  Installation_zone 35
8/8 [==============================] - 0s 3ms/step
Installation_zone 29
3/3 [==============================] - 0s 5ms/step
Installation_zone 22
6/6 [==============================]